In [2]:
import sys
import pylab
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


from _util import * 
import matplotlib.pyplot as plt
## make ohio into an iterative environment
import simulation.Ohio_Simulator as Ohio
reload(Ohio)


1. that tf_1 why does not work?
2. this why slow?
3. PG how to be parallel?
2. T = 24
3. lr

In [7]:
# In this example, we use REINFORCE algorithm which uses monte-carlo update rule
class REINFORCEAgent():
    def __init__(self, state_size, action_size, hid_dim = 32, gamma = 0.8, lr = 0.0002):
        # if you want to see Cartpole learning, then change to True
        self.render = False
        self.load_model = False
        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size

        # These are hyper parameters for the Policy Gradient
        self.discount_factor = gamma
        self.learning_rate = lr
        self.hidden1, self.hidden2 = hid_dim, hid_dim

        # create model for policy network
        self.model = self.build_model()

        # lists for the states, actions and rewards
        self.states, self.actions, self.rewards = [], [], []

    # approximate policy using Neural Network
    # state is input and probability of each action is output of network
    def build_model(self):
        model = Sequential()
        model.add(Dense(self.hidden1, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_uniform'))
        # model.add(Dense(self.hidden2, activation='relu', kernel_initializer='glorot_uniform'))
        model.add(Dense(self.action_size, activation='softmax', kernel_initializer='glorot_uniform'))
        model.summary()
        # Using categorical crossentropy as a loss is a trick to easily
        # implement the policy gradient. Categorical cross entropy is defined
        # H(p, q) = sum(p_i * log(q_i)). For the action taken, a, you set 
        # p_a = advantage. q_a is the output of the policy network, which is
        # the probability of taking the action a, i.e. policy(s, a). 
        # All other p_i are zero, thus we have H(p, q) = A * log(policy(s, a))
        model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=self.learning_rate))
        return model

    # using the output of policy network, pick action stochastically
    def get_action(self, state):
        policy = self.model.predict(state, batch_size=1).flatten()
        return np.random.choice(self.action_size, 1, p=policy)[0]

    # In Policy Gradient, Q function is not available.
    # Instead agent uses sample returns for evaluating policy
    def discount_rewards(self, rewards):
        discounted_rewards = np.zeros_like(rewards)
        running_add = 0
        for t in reversed(range(0, len(rewards))):
            running_add = running_add * self.discount_factor + rewards[t]
            discounted_rewards[t] = running_add
        return discounted_rewards

    # save <s, a ,r> of each step
    def append_sample(self, state, action, reward):
        self.states.append(state)
        self.rewards.append(reward)
        self.actions.append(action)

    # update policy network every episode
    def train_model(self):
        episode_length = len(self.states)

        discounted_rewards = self.discount_rewards(self.rewards)
        discounted_rewards -= np.mean(discounted_rewards)
        discounted_rewards /= np.std(discounted_rewards)

        update_inputs = np.zeros((episode_length, self.state_size))
        advantages = np.zeros((episode_length, self.action_size))

        for i in range(episode_length):
            update_inputs[i] = self.states[i]
            advantages[i][self.actions[i]] = discounted_rewards[i]

        self.model.fit(update_inputs, advantages, epochs=1, verbose=0)
        self.states, self.actions, self.rewards = [], [], []


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
state_size = 15
action_size = 5
# make REINFORCE agent
agent = REINFORCEAgent(state_size = state_size, action_size = action_size
                       , hid_dim = 32, gamma = 0.8, lr = 0.0002)

gamma = 0.8
lr = 0.002
n_epoch = 10000
T = 24
e_values = zeros(n_epoch)
e_average_values = zeros(n_epoch)
k = 0
EPISODES = 1000
env = Ohio.OhioSimulator(sd_G = 3, T = 24, N = 1000)

scores, episodes = [], []
disc_scores = zeros(EPISODES)
t_get_A = 0
t_step = 0
recorders = {}
for e in range(EPISODES):
    done = False
    score = 0
    state = env.reset(T)
    state = np.reshape(state, [1, state_size])
    print(e)
    running_rewards = []
    recorders[e] = {"state" : [], "action" : [], "reward" : []}
    while not done:

        # get action for the current state and go one step in environment
        curr = now()
        action = agent.get_action(state)
        t_get_A += now() - curr; curr = now()
        next_state, reward, done = env.online_step(action)
        t_step += now() - curr; curr = now()
        next_state = np.reshape(next_state, [1, state_size])
        
        recorders[e]["state"].append(state)
        recorders[e]["action"].append(action)
        recorders[e]["reward"].append(reward)
        
        running_rewards.append(reward)
        gammas = arr([gamma ** t for t in range(len(running_rewards))])
        disc_scores[e] = np.sum(running_rewards * gammas)
        
        # save the sample <s, a, r> to the memory
        agent.append_sample(state, action, reward)

        score += reward
        state = next_state

        if done:
            # every episode, agent learns from sample returns
            agent.train_model()

            # every episode, plot the play time
            score = score if score == 500 else score + 100
            scores.append(score)
            episodes.append(e)
            
            if e > 10 and e < 100:
                print("{} DONE with value {}, outlier = {}".format(e, np.mean(disc_scores[(e - 10):e]), np.min(disc_scores[(e - 10):e])))
            elif e > 100:
                print("{} DONE with value {}, outlier = {}".format(e, np.mean(disc_scores[(e - 100):e]), np.min(disc_scores[(e - 100):e]) ))

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 32)                512       
_________________________________________________________________
dense_13 (Dense)             (None, 5)                 165       
Total params: 677
Trainable params: 677
Non-trainable params: 0
_________________________________________________________________
0
1
2
3
4
5
6
7
8
9
10
11
11 DONE with value -49.91017127128628, outlier = -97.20314621923531
12
12 DONE with value -43.539617147193425, outlier = -97.20314621923531
13
13 DONE with value -40.477361508126336, outlier = -97.20314621923531
14
14 DONE with value -35.88315257343386, outlier = -97.20314621923531
15
15 DONE with value -33.23922730473152, outlier = -97.20314621923531
16
16 DONE with value -31.79752629081217, outlier = -97.20314621923531
17
17 DONE with value -30.083335795345562, outlier = -97.2031462192

In [12]:
policy = agent.model.predict(state, batch_size=1).flatten()
# return np.random.choice(agent.action_size, 1, p=policy)[0]


In [13]:
policy

array([nan, nan, nan, nan, nan], dtype=float32)

In [15]:
agent.model.predict(randn(10,15))

array([[nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan]], dtype=float32)

In [40]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# make REINFORCE agent
agent = REINFORCEAgent(state_size = 15, action_size = 5
                       , hid_dim = 32, gamma = 0.8, lr = 0.0002)

scores, episodes = [], []
disc_scores = zeros(EPISODES)

for e in range(EPISODES):
    done = False
    score = 0
    state = env.reset(T)
    state = np.reshape(state, [1, state_size])
    print(e)
    running_rewards = []
    while not done:

        # get action for the current state and go one step in environment
        action = agent.get_action(state)
        next_state, reward, done = env.online_step(action)
        next_state = np.reshape(next_state, [1, state_size])
        
        running_rewards.append(reward)
        gammas = arr([gamma ** t for t in range(len(running_rewards))])
        disc_scores[e] = np.sum(running_rewards * gammas)
        
        # save the sample <s, a, r> to the memory
        agent.append_sample(state, action, reward)

        score += reward
        state = next_state

        if done:
            # every episode, agent learns from sample returns
            agent.train_model()

            # every episode, plot the play time
            score = score if score == 500 else score + 100
            scores.append(score)
            episodes.append(e)
            
            if e > 10 and e < 100:
                print("{} DONE with value {}".format(e, np.mean(disc_scores[(e - 10):e])))
            elif e > 100:
                print("{} DONE with value {}".format(e, np.mean(disc_scores[(e - 100):e])))

Physical devices cannot be modified after being initialized
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 32)                512       
_________________________________________________________________
dense_18 (Dense)             (None, 5)                 165       
Total params: 677
Trainable params: 677
Non-trainable params: 0
_________________________________________________________________
0
1
2
3
4
5
6
7
8
9
10
11
11 DONE with value -65.23588493085438
12
12 DONE with value -39.993993765910545
13
13 DONE with value -33.94995048878401
14
14 DONE with value -30.306916311065983
15
15 DONE with value -32.20486063536789
16
16 DONE with value -22.54349975761431
17
17 DONE with value -25.341362669697055
18
18 DONE with value -26.763269863008453
19
19 DONE with value -26.253181005485732
20
20 DONE with value -34.29210484527061
21
21 DONE with valu